## Cleaning the survey data

In cleaning the survey data, we focused on certain questions from the survey that we felt were most important for our research question. So, we first cut the number of columns we have from 550 to about 92. 

Then, we have to replace some 'NaN' entries with zeros, doing so only if respondents answers at least one question from that question bunch. For example, for questions `S15r1-r11` which are self reported sports obsession, if a respondent answers at least one of those with a value, we turned other 'NaN' values to 0. If the respondent left ALL entries to those questions blank, we dropped that row. This resulted in dropping about 3000 surveys from the data set. 

Beyond that, we create two new columns based on `S15` and `TEAM6` survey questions by viewing these question responses as a vector and finding the Euclidean norm. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('data/nfl_survey_data.xlsx')
df.shape

In [ ]:
## From the survey data, we see there are 550 columns.
## We want to include only the following features for initial
## exploration.

## these features are mainly demographic features
features = ['S1', 'S2', 'D4','D5','D6','Hid_Ethnicity_Buckets','Hid_Age','S12r3','VL2','NFL3','NFL4']

## self reported fan obsession
S15 = ['S15r' + str(i) for i in range(1,12)]

##
S13 = ['S13r1','S13r2','S13r3' ,'S13r4']

## these are our output variables
VL1 = ['VL1r' + str(i) for i in range(1,16)]

## 
VL3 = ['VL3r1','VL3r2','VL3r3','VL3r4','VL3r5','VL3r6']

##
VL4 =['VL4r1','VL4r2','VL4r3','VL4r4','VL4r5','VL4r6']

##
TEAM6 = ['TEAM6r' + str(i) for i in range(1,33)]

##
NFL1 = ['NFL1r' + str(i) for i in range(1,5)]
NFL2 = ['NFL2r' + str(i) for i in range(1,6)]

## creating one array of features of interest
features = features + S15 + S13 + VL1 + VL3 + VL4 + TEAM6 + NFL1 + NFL2

## new dataframe with only the features from above
df = df[features]

In [ ]:
## for the survey responses (target columns), if ALL entries for those questions
## are null, we drop that row. If respondents answered at least one question from that 
## set of questions, we turn null values to '0'. We do this for 

targets = S15  
targ_df = 1*~df[targets].isnull() # finding all null values, negating T/F values, and turning to 0/1's
targ_df['sum'] = targ_df.sum(axis = 1) # finding the sum of all rows from the 0/1 df above
indices = targ_df.loc[targ_df['sum'] == 0].index #finding those indices for which sum is 0
df_clean = df.drop(indices) #dropping those rows with ALL 0 entries. 

## same process as S15 for below
targets = S13
targ_df = 1*~df_clean[targets].isnull()
targ_df['sum'] = targ_df.sum(axis = 1)
indices = targ_df.loc[targ_df['sum'] == 0].index
df_clean = df_clean.drop(indices)

targets = VL3
targ_df = 1*~df_clean[targets].isnull()
targ_df['sum'] = targ_df.sum(axis = 1)
indices = targ_df.loc[targ_df['sum'] == 0].index
df_clean = df_clean.drop(indices)

targets = VL4
targ_df = 1*~df_clean[targets].isnull()
targ_df['sum'] = targ_df.sum(axis = 1)
indices = targ_df.loc[targ_df['sum'] == 0].index
df_clean = df_clean.drop(indices)

targets = TEAM6
targ_df = 1*~df_clean[targets].isnull()
targ_df['sum'] = targ_df.sum(axis = 1)
indices = targ_df.loc[targ_df['sum'] == 0].index
df_clean = df_clean.drop(indices)

df_clean = df_clean.fillna(0)

In [ ]:
## Creating magnitude columns
## we take the questions from S15 (S15r1-r11) which ask a respondent to self report
## fan obsession. We then form a column of the euclidean norm of the vector of responses
## from the S15 questions. 

col_sum = []
for i,col in enumerate(S15):
    print('working on column ', col)
    ## if we are on S15r1, we square the column 
    if i == 0:
        col_sum = df_clean[col].values**2
    ## after the first question, we now sum the squares
    else: 
        col_sum = df_clean[col]**2 + col_sum

## we want to form a new dataframe, which is the sum of the squares
df_sum = pd.DataFrame(col_sum)

## we square root the sum of squares dataframe, giving us the magnitude.
df_sum = df_sum.apply(np.sqrt)

## taking the magnitude dataframe, which is a single column, we place that
## in the clean dataframe under correct column name. 
df_clean['Fan_magnitude'] = df_sum

###### 
######

## we want to reverse the scale for the Team6 questions, where 4 is now the highest value, 
## and 1 the lowest value. 

for team in TEAM6:
#     print('working on team question:', team)
    team_list = df_clean[team].values
    new_list = list()
    
    for i in range(len(team_list)):
        if team_list[i] == 4:
            new_list.append(1)
        if team_list[i] == 3:
            new_list.append(2)
        if team_list[i] == 2:
            new_list.append(3)
        if team_list[i] == 1:
            new_list.append(4)
        if team_list[i] == 0:
            new_list.append(0)
#     print(team_list[:15])
    df5 = pd.DataFrame({'xs':new_list})
    df_clean[team] = df5['xs'].values
    
###### 
######

## this magnitude column is formed similar to what was done above with the S15 questions. 

col_sum = []
for i,col in enumerate(TEAM6):
    print('working on column ', col)
    if i == 0:
        col_sum = df_clean[col].values**2
    else: 
        col_sum = df_clean[col]**2 + col_sum

df_sum = pd.DataFrame(col_sum)
df_sum = df_sum.apply(np.sqrt)

df_clean['Team6_magnitude'] = df_sum

In [ ]:
df_clean.to_csv('sports_survey_clean.csv', index = False)